# Дополнительное задание:
Вы справились с исследованием данных! Ваша стажировка продолжается, в мире произошли непредвиденные события — пандемия COVID-19. Вам доверена срочная задача. Чтобы сформировать понимание о стратегии компании, необходимо определить возможные сценарии дальнейшего развития событий.

In [1]:
import numpy as np # Numerical Python
import pandas as pd # работа с данными
import matplotlib.pyplot as plt # визуализация
import seaborn as sns
from scipy import stats # статистика

### 11	Загрузите файл dip_hw_covid.csv в pandas dataframe

Датасет содержит информацию о распространении COVID-19 в России и США:

In [4]:
my_path = r'data/dip_hw_covid.csv'
df = pd.read_csv(my_path)
df.head()

,date,country,confirmed,deaths
0,2020-01-22 00:00:00,Russia,0.0,0.0
1,2020-01-22 00:00:00,US,1.0,0.0
2,2020-01-23 00:00:00,Russia,0.0,0.0
3,2020-01-23 00:00:00,US,1.0,0.0
4,2020-01-24 00:00:00,Russia,0.0,0.0


Значение столбцов:

- date — дата фиксации значений;
- country — страна;
- confirmed — заражений зафиксировано;
- deaths — смертей зафиксировано.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1120 entries, 0 to 1119
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       1116 non-null   object 
 1   country    1116 non-null   object 
 2   confirmed  1116 non-null   float64
 3   deaths     1116 non-null   float64
dtypes: float64(2), object(2)
memory usage: 35.1+ KB


In [8]:
df.describe()

,confirmed,deaths
count,1.116000e+03,1116.000000
mean,8.508892e+06,169543.963262
std,1.130533e+07,198133.420131
min,0.000000e+00,0.000000
25%,8.245898e+05,15199.000000
50%,3.840895e+06,98314.000000
75%,8.748694e+06,226763.000000
max,3.500342e+07,613228.000000


In [6]:
df['country'].value_counts()

country
Russia    558
US        558
Name: count, dtype: int64

Данные без пропусков, количество записаей одинаково для России и США. 

Количество зафиксированных заражений от 0 до 35 003 420 в день.

Количество смертей от 0 до 613 228 в день.

### 12	Исследуйте данные: 
      1. Рассчитайте коэффициент корреляции между заболеваемостью и смертностью; 
      2. Визуализируйте динамики заболеваемости и смертности по доступным странам в отдельности, сравните характер распространения	



### 13	Постройте модель линейной регрессии по смертям от заболеваемости и вычислите прогноз количества смертей в России для 25 000 000 случаев заражений 
(примечание: сформируйте выборку по России, выделите признак и целевую переменную [используем 100% данных для обучения], постройте прогноз смертей для 25 000 000 заболевших [поможет метод predict], сделайте выводы по полученному прогнозу)
### 14	Дополните шаблон презентации отдельными слайдами с информацией по построению модели, получению прогноза и рекомендациями для бизнеса с учётом полученной информации (создайте разделы: “Построение модели” и “Рекомендации для бизнеса”).